# I Fäders spår för framtids sägrar!
using Sam's API
https://github.com/basegpu/vasapi/blob/main/readme.md

input is year, sex and rank (! not bib)

In [111]:
import requests
import json
import pandas as pd
import time
import pickle
import datetime

## example: one request

In [5]:
r = requests.get('https://vasa-api.herokuapp.com/result/2022/M/2999')
print(r.status_code)
print(r.headers)
print(r.text)

200
{'Connection': 'keep-alive', 'Server': 'gunicorn/19.10.0', 'Date': 'Sat, 12 Mar 2022 09:50:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '178', 'Via': '1.1 vegur'}
{"Lopper":{"Bib":"4946","Group":"H21","Name":"Gustafsson, Olof","Nation":"SWE","Sex":"M"},"Overall":{"Place":3161,"StartGroup":"VL4","Time":"06:07:52"},"Place":2999,"Year":2022}



# get data

In [135]:
def get_data_from_api(year, sex, nr_start, nr_end, pickling=False):
    print('------------------------------------------')
    print(f'INFO: started at: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
    print(f'--- {nr_start} to {nr_end} ---')
    t_start = time.time()
    
    i = nr_start-1 # start prior, add 1 at beginning of loop, because of 'continue' in loop
    racers = []
    
    while i < nr_end:
        i = i+1
        r = requests.get(f'https://vasa-api.herokuapp.com/result/{year}/{sex}/{i}')
        if r.status_code != 200:
            print(f'WARNING: bib not found: {i}')
            print(f'WARNING: Status Code: {r.status_code}')
            continue #stop current iteration
        
        # json2dict
        data = json.loads(r.text)
        # dict to flat list
        racer = list(data['Lopper'].values())+list(data['Overall'].values())+[data['Place'],data['Year']]
        # add to results
        racers = racers + [racer]
        
        if ((i-nr_start) == (int((nr_end-nr_start)/4))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} 1/4 way there ... {i}, {int(time.time()-t_start)}s, ') 
        if ((i-nr_start) == (int((nr_end-nr_start)/2))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} half way there ... {i}, {int(time.time()-t_start)}s, ') 
        if ((i-nr_start) == (int((nr_end-nr_start)*3/4))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} 3/4 way there ... {i}, {int(time.time()-t_start)}s, ') 
        
        
    # define the columns names from the keys
    keys = list(data['Lopper'].keys())+list(data['Overall'].keys())+['Place_cat','Year']
    
    # create df
    df = pd.DataFrame(racers, columns=keys)
    elapsed_time = time.time()-t_start
    print(f'INFO: elapsed time {int(elapsed_time)}s (= {int(elapsed_time/60)}min)')
    print(f'INFO: records: {df.shape[0]} from possible {nr_end-nr_start+1}')
    
    # pickle for framtids sägrer
    if pickling:
        print(f'INFO: pickling ...')
        filename = f'data/results_{year}_{sex}_{nr_start}_{nr_end}.pickle'
        with open(filename, 'wb') as f:
            pickle.dump(df, f)
        print(f'INFO: ... DONE: filename {filename}')
    return(df)


In [128]:

df = get_data_from_api(2022,'M',1001,2000, pickling=True)


INFO: started at: 12/03/2022 12:26:32
INFO: half way there ... 1500
INFO: elapsed time 1214s (= 20min)
INFO: records: 1000 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_1001_2000.pickle


In [133]:
df = get_data_from_api(2022,'M',2001,4000, pickling=True)


INFO: started at: 12/03/2022 14:28:21
INFO: 12/03/2022 14:30:10 1/4 way there ... 2500, 108s, 
INFO: 12/03/2022 14:40:50 half way there ... 3000, 748s, 
INFO: 12/03/2022 14:51:47 3/4 way there ... 3500, 1405s, 
INFO: elapsed time 1995s (= 33min)
INFO: records: 1999 from possible 2000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_2001_4000.pickle


In [134]:
df = get_data_from_api(2022,'M',4001,5000, pickling=True)


INFO: started at: 12/03/2022 15:02:04
INFO: 12/03/2022 15:07:29 1/4 way there ... 4250, 325s, 
INFO: 12/03/2022 15:12:22 half way there ... 4500, 617s, 
INFO: 12/03/2022 15:17:22 3/4 way there ... 4750, 918s, 
INFO: elapsed time 1246s (= 20min)
INFO: records: 998 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_4001_5000.pickle


In [137]:
# DO IT FOR MANY, SAFE EVERY 1000 RACERS IN A FILE IN CASE OF CONNECTION TROUBLE

for i in [5000,6000,7000,8000,9000,10_000,11_000,12_000,13_000,14_000,15_000,16_000,17_000]:
    df = get_data_from_api(2022,'M',i+1,i+1000, pickling=True)


------------------------------------------
INFO: started at: 12/03/2022 15:26:19
--- 5001 to 6000 ---
INFO: 12/03/2022 15:31:01 1/4 way there ... 5250, 281s, 
INFO: 12/03/2022 15:36:00 half way there ... 5500, 580s, 
INFO: 12/03/2022 15:40:54 3/4 way there ... 5750, 874s, 
INFO: elapsed time 1199s (= 19min)
INFO: records: 999 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_5001_6000.pickle
------------------------------------------
INFO: started at: 12/03/2022 15:46:18
--- 6001 to 7000 ---
INFO: 12/03/2022 15:51:27 1/4 way there ... 6250, 308s, 
INFO: 12/03/2022 15:56:25 half way there ... 6500, 606s, 
INFO: 12/03/2022 16:01:32 3/4 way there ... 6750, 913s, 
INFO: elapsed time 1256s (= 20min)
INFO: records: 999 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_6001_7000.pickle
------------------------------------------
INFO: started at: 12/03/2022 16:07:15
--- 7001 to 8000 ---
INFO: 12/03/2022 16:12:22 1/4 way there ..

INFO: elapsed time 612s (= 10min)
INFO: records: 56 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_10001_11000.pickle
------------------------------------------
INFO: started at: 12/03/2022 17:21:45
--- 11001 to 12000 ---


UnboundLocalError: local variable 'data' referenced before assignment